In [6]:
# Importul bibliotecilor necesare pentru inregistrare audio, manipulare fisiere si comunicare cu Azure Speech Service
import pyaudio
import wave
import os
import keyboard
from pydub import AudioSegment
import sys
import requests
import json

print("Biblioteci incarcate cu succes.")

# ---------------------------- Configuratie ----------------------------

def load_config(config_path='config.json'):
    """
    Incarca fisierul config.json ce contine credentialele pentru Azure Speech Service.
    Returneaza un dictionar cu datele de configurare sau None daca fisierul lipseste/sunt erori.
    """
    try:
        with open(config_path, 'r') as config_file:
            config = json.load(config_file)
        return config
    except FileNotFoundError:
        print(f"Fisierul de configuratie '{config_path}' nu a fost gasit!")
        return None
    except json.JSONDecodeError:
        print(f"JSON invalid in fisierul de configuratie '{config_path}'")
        return None

# Incarca credentialele din fisierul JSON
config = load_config()
if config is None:
    sys.exit(1)

# Extrage valorile necesare pentru Azure Speech din configuratie
AZURE_SPEECH_KEY = config.get('speech_subscription_key') or config.get('subscription_key')
AZURE_SPEECH_REGION = config.get('speech_region') or config.get('region')
AZURE_SPEECH_ENDPOINT = config.get('speech_endpoint') or config.get('endpoint')

# ---------------------------- Inregistrare Audio ----------------------------

def record_audio(output_file="recorded_audio.mp3"):
    """
    Inregistreaza sunet de la microfon pana cand utilizatorul apasa tasta 'q'.
    Salveaza fisierul rezultat in format MP3.
    """
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("Se inregistreaza... Apasa 'q' pentru a opri.")
    frames = []

    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)
        if keyboard.is_pressed("q"):
            print("Inregistrarea s-a oprit.")
            break

    stream.stop_stream()
    stream.close()
    p.terminate()

    try:
        # Salveaza temporar ca WAV, apoi converteste in MP3
        temp_wav = "temp_audio.wav"
        wf = wave.open(temp_wav, "wb")
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b"".join(frames))
        wf.close()

        audio = AudioSegment.from_file(temp_wav)
        audio.export(output_file, format="mp3")
        os.remove(temp_wav)
        print(f"Inregistrarea salvata in {output_file}")
        return output_file
    except Exception as e:
        print(f"Eroare la salvarea audio: {e}")
        if os.path.exists(temp_wav):
            os.remove(temp_wav)
        return None

# ---------------------------- Conversii Audio ----------------------------

def convert_mp4_to_mp3(input_file, output_file="converted_audio.mp3"):
    """
    Converteste un fisier video MP4 in fisier audio MP3.
    """
    try:
        audio = AudioSegment.from_file(input_file)
        audio.export(output_file, format="mp3")
        print(f"S-a convertit {input_file} in {output_file}")
        return output_file
    except Exception as e:
        print(f"Eroare la convertirea MP4 in MP3: {e}")
        return None

def split_audio_to_chunks(audio_path, chunk_length_ms=60000):
    """
    Imparte fisierul audio in bucati de lungime egala (default 60 secunde).
    Returneaza o lista cu numele fisierelor temporare WAV.
    """
    audio = AudioSegment.from_file(audio_path)
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        chunk_filename = f"chunk_{i // chunk_length_ms}.wav"
        chunk = chunk.set_channels(1).set_frame_rate(16000)
        chunk.export(chunk_filename, format="wav")
        chunks.append(chunk_filename)
    return chunks

# ---------------------------- Transcriere cu Azure ----------------------------

def transcribe_audio_azure(audio_file, language="en-US"):
    """
    Transcrie un fisier audio folosind Azure Speech Service.
    Daca fisierul e lung, il imparte in bucati de 60 secunde si concateneaza rezultatele.
    """
    try:
        endpoint = f"https://{AZURE_SPEECH_REGION}.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1"
        headers = {
            'Ocp-Apim-Subscription-Key': AZURE_SPEECH_KEY,
            'Content-Type': 'audio/wav; codecs=audio/pcm; samplerate=16000',
            'Accept': 'application/json'
        }
        params = {
            'language': language,
            'format': 'detailed'
        }

        chunks = split_audio_to_chunks(audio_file)
        full_transcript = ""

        for chunk_path in chunks:
            with open(chunk_path, 'rb') as audio_data:
                response = requests.post(endpoint, headers=headers, params=params, data=audio_data)

            os.remove(chunk_path)

            if response.status_code == 200:
                result = response.json()
                if result.get('RecognitionStatus') == 'Success':
                    nbest = result.get('NBest')
                    if nbest:
                        full_transcript += nbest[0].get('Display', '') + "\n"
                    else:
                        full_transcript += result.get('DisplayText', '') + "\n"
                else:
                    print(f"Eroare la recunoastere: {result.get('RecognitionStatus')}")
            else:
                print(f"Eroare HTTP {response.status_code}: {response.text}")

        return full_transcript.strip()

    except Exception as e:
        print(f"Eroare la transcrierea audio cu Azure Speech Service: {e}")
        return None

# ---------------------------- Utilitare ----------------------------

def get_language_code(lang_input):
    """
    Transforma o limba scrisa de utilizator intr-un cod standard compatibil Azure.
    """
    language_map = {
        "en": "en-US",
        "ro": "ro-RO",
        "english": "en-US",
        "engleza": "en-US",
        "romanian": "ro-RO",
        "romana": "ro-RO"
    }
    return language_map.get(lang_input.lower(), "en-US")

def get_next_transcript_filename(base_dir="TranscriptsAzure"):
    """
    Genereaza un nume unic pentru fisierul de transcript.
    Creeaza folderul 'TranscriptsAzure' daca nu exista.
    """
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    base_filename = "transcript.txt"
    filepath = os.path.join(base_dir, base_filename)

    if not os.path.exists(filepath):
        return filepath

    counter = 1
    while True:
        filename = f"transcript_{counter}.txt"
        filepath = os.path.join(base_dir, filename)
        if not os.path.exists(filepath):
            return filepath
        counter += 1

# ---------------------------- Program principal ----------------------------

if not AZURE_SPEECH_KEY or not AZURE_SPEECH_REGION:
    print(" Lipsesc credentialele Azure Speech Service din config.json!")
    sys.exit(1)

print("Alege o optiune:")
print("1. Inregistreaza audio nou")
print("2. Transcrie fisier MP3 existent")
print("3. Converteste MP4 in MP3 si transcrie")
choice = input("Introdu 1, 2 sau 3: ")

if choice == "1":
    audio_file = record_audio()
    if audio_file:
        lang = input("Introdu limba (en pentru engleza, ro pentru romana, sau lasa gol pentru engleza): ")
        azure_lang = get_language_code(lang or "en")
        text = transcribe_audio_azure(audio_file, language=azure_lang)
        if text:
            transcript_file = get_next_transcript_filename()
            with open(transcript_file, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"Transcrierea salvata in {transcript_file}")

elif choice == "2":
    file_path = input("Introdu calea catre fisierul MP3: ")
    if not os.path.exists(file_path):
        print("Fisierul nu a fost gasit!")
    elif not file_path.lower().endswith(".mp3"):
        print("Te rog sa furnizezi un fisier MP3!")
    else:
        lang = input("Introdu limba (en pentru engleza, ro pentru romana, sau lasa gol pentru engleza): ")
        azure_lang = get_language_code(lang or "en")
        text = transcribe_audio_azure(file_path, language=azure_lang)
        if text:
            transcript_file = get_next_transcript_filename()
            with open(transcript_file, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"Transcrierea salvata in {transcript_file}")

elif choice == "3":
    file_path = input("Introdu calea catre fisierul MP4: ")
    if not os.path.exists(file_path):
        print("Fisierul nu a fost gasit!")
    elif not file_path.lower().endswith(".mp4"):
        print("Te rog sa furnizezi un fisier MP4!")
    else:
        mp3_file = convert_mp4_to_mp3(file_path)
        if mp3_file:
            lang = input("Introdu limba (en pentru engleza, ro pentru romana, sau lasa gol pentru engleza): ")
            azure_lang = get_language_code(lang or "en")
            text = transcribe_audio_azure(mp3_file, language=azure_lang)
            if text:
                transcript_file = get_next_transcript_filename()
                with open(transcript_file, "w", encoding="utf-8") as f:
                    f.write(text)
                print(f"Transcrierea salvata in {transcript_file}")
else:
    print("Optiune invalida!")


Biblioteci incarcate cu succes.
Alege o optiune:
1. Inregistreaza audio nou
2. Transcrie fisier MP3 existent
3. Converteste MP4 in MP3 si transcrie
Transcrierea salvata in Transcripts\transcript_3.txt
